In [1]:
import os
import copy
import dota2api
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

%matplotlib inline

In [2]:
API_KEY = '2029A39ECAAB627B5F71CF4C3E30F72D'
api = dota2api.Initialise(API_KEY)

In [3]:
data = pd.read_csv('../data/draft/capmodedata.csv')

In [4]:
data.head(3)

,match_id,radiant_win,duration,1is_pick,1team_id,1hero_id,2is_pick,2team_id,2hero_id,3is_pick,...,17hero_id,18is_pick,18team_id,18hero_id,19is_pick,19team_id,19hero_id,20is_pick,20team_id,20hero_id
0,1928898739,True,2885,False,0,73,False,1,69,False,...,41.0,False,0.0,81.0,True,1.0,105.0,True,0.0,94.0
1,1928932285,True,1894,False,0,69,False,1,102,False,...,40.0,False,0.0,28.0,True,1.0,97.0,True,0.0,62.0
2,1928903165,True,2424,False,1,85,False,0,68,False,...,30.0,False,1.0,47.0,True,0.0,58.0,True,1.0,33.0


In [5]:
temp = data.loc[data['1team_id'] == 0]

In [6]:
win1 = temp.loc[temp['radiant_win'] == False]

In [7]:
win1.shape

(111599, 63)

In [8]:
temp2 = data.loc[data['1team_id'] == 1]

In [9]:
win2 = temp2.loc[temp2['radiant_win'] == True]
win2.shape

(115713, 63)

In [113]:
win1

,match_id,radiant_win,duration,1is_pick,1team_id,1hero_id,2is_pick,2team_id,2hero_id,3is_pick,...,17hero_id,18is_pick,18team_id,18hero_id,19is_pick,19team_id,19hero_id,20is_pick,20team_id,20hero_id
8,1928902687,False,2595,False,0,75,False,1,57,False,...,39.0,False,0.0,11.0,True,1.0,63.0,True,0.0,21.0
9,1928917475,False,2019,False,0,69,False,1,85,False,...,86.0,False,0.0,20.0,True,1.0,26.0,True,0.0,87.0
10,1928908270,False,2329,False,0,73,False,1,1,False,...,71.0,False,0.0,86.0,True,1.0,47.0,True,0.0,49.0
14,1928897896,False,2841,False,0,69,False,1,91,False,...,75.0,False,0.0,1.0,True,1.0,36.0,True,0.0,55.0
15,1928914863,False,2380,False,0,100,False,1,73,False,...,44.0,False,0.0,41.0,True,1.0,67.0,True,0.0,106.0
25,1928882302,False,3297,False,0,12,False,1,73,False,...,32.0,False,0.0,35.0,True,1.0,30.0,True,0.0,6.0
28,1928898323,False,2830,False,0,32,False,1,12,False,...,70.0,False,0.0,47.0,True,1.0,81.0,True,0.0,59.0
34,1928922418,False,2250,False,0,73,False,1,69,False,...,112.0,False,0.0,67.0,True,1.0,71.0,True,0.0,70.0
35,1928899945,False,3131,False,0,69,False,1,106,False,...,29.0,False,0.0,36.0,True,1.0,82.0,True,0.0,81.0
39,1928918024,False,2016,False,0,39,False,1,75,False,...,4.0,False,0.0,61.0,True,1.0,74.0,True,0.0,28.0


In [11]:
111599+115713

227312

In [12]:
win2.head(4)

,match_id,radiant_win,duration,1is_pick,1team_id,1hero_id,2is_pick,2team_id,2hero_id,3is_pick,...,17hero_id,18is_pick,18team_id,18hero_id,19is_pick,19team_id,19hero_id,20is_pick,20team_id,20hero_id
2,1928903165,True,2424,False,1,85,False,0,68,False,...,30.0,False,1.0,47.0,True,0.0,58.0,True,1.0,33.0
3,1928907204,True,2588,False,1,71,False,0,73,False,...,30.0,False,1.0,68.0,True,0.0,50.0,True,1.0,21.0
4,1928865084,True,3658,False,1,12,False,0,42,False,...,93.0,False,1.0,67.0,True,0.0,72.0,True,1.0,40.0
6,1928926702,True,1998,False,1,85,False,0,75,False,...,9.0,False,1.0,68.0,True,0.0,43.0,True,1.0,23.0


In [13]:
to_concat_dataframes = [win1, win2]
all_win_data_only = pd.concat(to_concat_dataframes)

In [14]:
all_win_data_only.sort_index()

,match_id,radiant_win,duration,1is_pick,1team_id,1hero_id,2is_pick,2team_id,2hero_id,3is_pick,...,17hero_id,18is_pick,18team_id,18hero_id,19is_pick,19team_id,19hero_id,20is_pick,20team_id,20hero_id
2,1928903165,True,2424,False,1,85,False,0,68,False,...,30.0,False,1.0,47.0,True,0.0,58.0,True,1.0,33.0
3,1928907204,True,2588,False,1,71,False,0,73,False,...,30.0,False,1.0,68.0,True,0.0,50.0,True,1.0,21.0
4,1928865084,True,3658,False,1,12,False,0,42,False,...,93.0,False,1.0,67.0,True,0.0,72.0,True,1.0,40.0
6,1928926702,True,1998,False,1,85,False,0,75,False,...,9.0,False,1.0,68.0,True,0.0,43.0,True,1.0,23.0
8,1928902687,False,2595,False,0,75,False,1,57,False,...,39.0,False,0.0,11.0,True,1.0,63.0,True,0.0,21.0
9,1928917475,False,2019,False,0,69,False,1,85,False,...,86.0,False,0.0,20.0,True,1.0,26.0,True,0.0,87.0
10,1928908270,False,2329,False,0,73,False,1,1,False,...,71.0,False,0.0,86.0,True,1.0,47.0,True,0.0,49.0
11,1928901534,True,2626,False,1,32,False,0,21,False,...,100.0,False,1.0,72.0,True,0.0,63.0,True,1.0,8.0
14,1928897896,False,2841,False,0,69,False,1,91,False,...,75.0,False,0.0,1.0,True,1.0,36.0,True,0.0,55.0
15,1928914863,False,2380,False,0,100,False,1,73,False,...,44.0,False,0.0,41.0,True,1.0,67.0,True,0.0,106.0


In [46]:
hero_1 = all_win_data_only['1hero_id']
heropick_1 = all_win_data_only['1is_pick']
hero_2 = all_win_data_only['2hero_id']
heropick_2 = all_win_data_only['2is_pick']
hero_3 = all_win_data_only['3hero_id']
heropick_3 = all_win_data_only['3is_pick']
hero_4 = all_win_data_only['4hero_id']
heropick_4 = all_win_data_only['4is_pick']
hero_5 = all_win_data_only['5hero_id']
heropick_5 = all_win_data_only['5is_pick']
hero_6 = all_win_data_only['6hero_id']
heropick_6 = all_win_data_only['6is_pick']

hero_7 = all_win_data_only['7hero_id']
# hero_8 = data['8hero_id']
# hero_9 = data['9hero_id']

In [47]:
test1 = pd.DataFrame(columns=['1hero_id','1is_pick' ,'2hero_id','2is_pick', '3hero_id','3is_pick', '4hero_id', '4is_pick', '5hero_id', '5is_pick','6hero_id', '6is_pick'])

In [48]:
test1['1hero_id'] = hero_1
test1['1is_pick'] = heropick_1
test1['2hero_id'] = hero_2
test1['2is_pick'] = heropick_2
test1['3hero_id'] = hero_3
test1['3is_pick'] = heropick_3
test1['4hero_id'] = hero_4
test1['4is_pick'] = heropick_4
test1['5hero_id'] = hero_5
test1['5is_pick'] = heropick_5
test1['6hero_id'] = hero_6
test1['6is_pick'] = heropick_6
# test1['7hero_id'] = hero_7
# test1['8hero_id'] = hero_8
# test1['9hero_id'] = hero_9

In [49]:
test1.head(3)

,1hero_id,1is_pick,2hero_id,2is_pick,3hero_id,3is_pick,4hero_id,4is_pick,5hero_id,5is_pick,6hero_id,6is_pick
8,75,False,57,False,41,False,73,False,26,True,5.0,True
9,69,False,85,False,112,False,28,False,7,True,100.0,True
10,73,False,1,False,69,False,63,False,87,True,72.0,True


In [50]:
X = test1.as_matrix()
X

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[75, False, 57, ..., True, 5.0, True],
       [69, False, 85, ..., True, 100.0, True],
       [73, False, 1, ..., True, 72.0, True],
       ...,
       [85, False, 89, ..., True, 87.0, True],
       [44, False, 29, ..., True, 18.0, True],
       [71, False, 100, ..., True, 11.0, True]], dtype=object)

In [51]:
X.shape

(227312, 12)

In [52]:
y = np.array(hero_7)

In [54]:
y.shape

(227312,)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
from sklearn.neural_network import MLPClassifier

In [58]:
model3 = MLPClassifier()

In [59]:
model3.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [60]:
print('Trainig is Finished')

Trainig is Finished


In [61]:
y_pred = model3.predict(X_test)

In [62]:
from sklearn.metrics import accuracy_score

In [63]:
accuracy_score(y_test, y_pred)

0.07271847436376834

In [64]:
np.unique(y_pred)

array([  5.,   7.,   8.,  12.,  16.,  20.,  21.,  25.,  26.,  29.,  30.,
        33.,  39.,  41.,  50.,  57.,  58.,  60.,  65.,  68.,  71.,  72.,
        75.,  81.,  85.,  86.,  99., 100., 112.])

In [33]:
# # Trying to do the roulette wheel
from sklearn import preprocessing

In [20]:
all_win_data_only = all_win_data_only.sort_index()

In [22]:
all_win_data_only.shape

(227312, 63)

In [23]:
hero_1 = all_win_data_only['1hero_id']

In [27]:
grouped1 = all_win_data_only.groupby(all_win_data_only['1hero_id']).size()

In [30]:
grouped1.head(3)

1hero_id
1    5620
2    1261
3     185
dtype: int64

In [35]:
# # Normalize the grouped1
minmaxscaler = preprocessing.MinMaxScaler()

In [38]:
group_minmax = minmaxscaler.fit_transform((grouped1.values).reshape(-1, 1))

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [48]:
np.sum(group_minmax)

5.70059714973906

In [58]:
grouped1.iloc[0]

5620

In [68]:
len(grouped1)

109

In [74]:
data_bucket = []
for i in range(len(grouped1)):
    count = grouped1.iloc[i]
    for j in range(count):
        data_bucket.append(count)

In [76]:
# # Shufling

In [81]:
np.random.shuffle(data_bucket)

In [94]:
np.random.choice(data_bucket)

39857

In [111]:
# # Getting the second ban (first ban for machine)
np.random.choice(data_bucket)

3289